**Springboard DSCT: 8.3 - SQL Case Study - PART 2: SQLite**

Initial Note: Errors on the PHPMyAdmin server resulted in Q9 being answered in this notebook. Q8 included for context.

QUESTIONS:

**Q8**: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than 30 Dollars. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

**Q9**: This time, produce the same result as in Q8, but using a subquery.

**Q10:** Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

**Q11:** Produce a report of members and who recommended them in alphabetic surname,firstname order


**Q12:** Find the facilities with their usage by member, but not guests


**Q13:** Find the facilities usage by month, but not guests

In [1]:
from sqlalchemy import *
import pandas as pd
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


C:\Users\Kstou\AppData\Local\Temp/ipykernel_29012/4197245412.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


In [2]:
with engine.connect() as con:
    r = con.execute('SELECT * FROM Bookings')
    B_df = pd.DataFrame(r.fetchall())
    B_df.columns = r.keys()

    print(B_df.head())

   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1


In [3]:
F_df = pd.read_sql_query('SELECT * FROM Facilities', engine)
print(F_df.head())

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  


In [4]:
M_df = pd.read_sql_query('SELECT * FROM Members', engine)
print(M_df.head())

   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  


**Running the Query from Q9:**

In [30]:
Question_9 = pd.read_sql_query("""SELECT firstname, surname, name AS Facility,
CASE WHEN memid = 0
THEN guestcost * slots
ELSE membercost * slots
END AS cost, starttime
FROM (SELECT * FROM Bookings AS b
JOIN Members AS m ON b.memid = m.memid
JOIN Facilities AS f ON b.facid = f.facid
WHERE starttime LIKE '2012-09-14%'
AND CASE WHEN m.memid = 0
THEN guestcost * slots
ELSE membercost * slots
END >30)
ORDER BY cost DESC;""", engine)
print(Question_9)

   firstname  surname        Facility   cost            starttime
0      GUEST    GUEST  Massage Room 2  320.0  2012-09-14 11:00:00
1      GUEST    GUEST  Massage Room 1  160.0  2012-09-14 09:00:00
2      GUEST    GUEST  Massage Room 1  160.0  2012-09-14 13:00:00
3      GUEST    GUEST  Massage Room 1  160.0  2012-09-14 16:00:00
4      GUEST    GUEST  Tennis Court 2  150.0  2012-09-14 17:00:00
5      GUEST    GUEST  Tennis Court 1   75.0  2012-09-14 16:00:00
6      GUEST    GUEST  Tennis Court 1   75.0  2012-09-14 19:00:00
7      GUEST    GUEST  Tennis Court 2   75.0  2012-09-14 14:00:00
8      GUEST    GUEST    Squash Court   70.0  2012-09-14 09:30:00
9     Jemima  Farrell  Massage Room 1   39.6  2012-09-14 14:00:00
10     GUEST    GUEST    Squash Court   35.0  2012-09-14 12:30:00
11     GUEST    GUEST    Squash Court   35.0  2012-09-14 15:00:00
